In [ ]:
## Bike Sharing Assignment
# import depended libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ignore warnings
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler 

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from sklearn.metrics import r2_score
### 1. Data Reading & Understanding
# Loading the loan data and converting into dataframe
df=pd.read_csv("day.csv")
df
df.shape
# To check if there is any headers/footers or summary details rows are there and delete it.
df.head() # summay about the loan Data
print(df.shape)
df.head()
df.tail() # checking if totaol, subtotoal rows present
# From above analysis We can see that there is no header/footers/summary/subtotal rows
# Also number of rows is 730 and columns is 16.

# To check if there is any null values.
df.info()
df.isnull().sum()
# We can see that there is no null values present in data.

# Here data looks with in the permissiable ranage.
df.describe()
#Delete  Unnessasry columns:
# 1. From above analysis we can see that the columns instant is unique in nature and act like a id which will not participate
#in analysis. Hence we can drop them.
# we can also see that only dtedat is of object type while others attribute is of int/float type.
# Since we already having month and year so we can drop dteday columns.

print("Total columns with values NA: ", len(df.columns[df.isna().all()].tolist()))

 #Lets delete all the columns which value is uniqe in nature.

unique_columns = [col for col in df.columns if df[col].nunique() == len(df)]
unique_columns
dropped_columns=['instant','dteday']
df.drop(dropped_columns,axis=1,inplace=True)
df.shape

# Checking  if there any duplicates row. 
duplicate_rows = len(df[df.duplicated()])
print("Count of duplicate rows:",round(duplicate_rows,2))
# No duplicaets rows found.

df.nunique().sort_values()
# so there is no any more columns with uniqe values.
# Inferences:
# We can see that values are in with the ranges. So need to remove any outliers.
# Transforming the Data
# Form data dictionary and abvoe analysis we can say that  season, weathersit & month are catogorical values but found as int.
#Lets Convert season, weathersit & month to Catogory type
df['season']=df['season'].astype('category')
df['weathersit']=df['weathersit'].astype('category')
df['mnth']=df['mnth'].astype('category')
df.info()
# As per data dictionary, following are the categorical variables has following values.
# 1. yr has two values 0 for 2018 and 1 for 2019.
# 2. holiday o or 1.
# 3. working day 0 or 1.
# 4. weathersit has three values good, bad and moderate weatherSit. 
# 5. season (1:spring, 2:summer, 3:fall, 4:winter)
# all the data seems to be in range
df.season.replace({1:"spring", 2:"summer", 3:"fall", 4:"winter"},inplace = True)
df.weathersit.replace({1:'good',2:'moderate',3:'bad',4:'severe'},inplace = True)

df.mnth = df.mnth.replace({1: 'jan',2: 'feb',3: 'mar',4: 'apr',5: 'may',6: 'jun',
                  7: 'jul',8: 'aug',9: 'sept',10: 'oct',11: 'nov',12: 'dec'})

df.weekday = df.weekday.replace({0: 'sun',1: 'mon',2: 'tue',3: 'wed',4: 'thu',5: 'fri',6: 'sat'})
df.head()
### 2. EDA
#### Univariate analysis
# Lets check if any outliers present is numeric featires.
# raw box plots for indepent variables with continuous values
cols = ['temp', 'atemp', 'hum', 'windspeed']
plt.figure(figsize=(18,4))

k = 1
for col in cols:
    plt.subplot(1,4,k)
    sns.boxplot(y=col, data=df)
    k+=1
# Line graph will that show the count vs month for year 2018 and 2019.
sns.lineplot(x = "mnth", y = "cnt", data=df,hue = "yr")
plt.show() 
# Inferences:
# 1. From Line Graph we can see that target Variable cnt has increased from 2018 to 2019.
# 2. Demand for bike is high between may to oct for year 2018 and 2019.
# 3. From Boxchart we can also see that there is no outliers present.
# Lets plot pairplots to have better to see if cnt is linearly  depend upon is independet variables. 
numerical_variabels=['cnt', 'temp', 'atemp', 'hum','windspeed']
sns.pairplot(data=df,vars=numerical_variabels, kind="reg")
plt.show()
# Inferences:
# 1. We can see that we have linealy dependecy between temp, atemp and count.
# 2. we can also see that temp and atemp is highly correlated.
####  Bivariate analysis
#  Barcharts for categorical variables to see demands
# function to create barplot related to categorical columns

def plot_bar_graphs(column):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    sns.barplot(x=column, y='cnt', data=df)
    plt.title(f'Bar Plot of {column} vs cnt')
    
    plt.subplot(1, 2, 2)
    sns.barplot(x=column, y='cnt', data=df, hue='yr', palette='Set1')
    plt.title(f'Bar Plot of {column} vs cnt (2018 vs 2019)')
    plt.legend(title='yr', labels=['2018', '2019'])
    
    plt.tight_layout()
    plt.show()
plot_bar_graphs('season')
# Inferences:
# 1. Fall  seasons is having high number bike rental.
# 2. Bike rental in 2019 has increaded for every seasion compared to 2018.

plot_bar_graphs('mnth')
# Inferences:
# 1. May to oct is having number of bike registration.
# 2. Bike registration in 2019 has increaded for every month compared to 2018.
plot_bar_graphs('weathersit')
# Inferences:
# 1. People prefer bike rental when weather situations is good. 
# 2. Bike registration in 2019 has increaded  for every weather situtation 2018.
plot_bar_graphs('weekday')
# Inferences:
# 1. People don't prefer bike rental on weekends. 

plot_bar_graphs('holiday')

# Inferences:
# 1. People dont prefer renting bike holiday.

plot_bar_graphs('workingday')
# Inferences:
# 1. People prefer bike rental on working days.
# Lets draw heatMap to indentify is there any correlation.

# Lets plot the corrlation matrix(heatmap)
plt.figure(figsize=(25,10))
sns.heatmap(df.corr(), cmap='BuGn', annot = True)
plt.show()
#Inferences from heatmap: 
#1. Since Count is sum of casual and registed. That we can infer from  heatmap as casual and registered are highly correleted with cnt. 
#2. holiday, hum, and windspeed is negatively correlated. 
#3. Indepedent variable temp and atemp is highly correlated(0.99).This show that there is high possiblity that one may have derived from another,One of the them can be dropped. Will use VIP and p values to drop this.
df.shape
# Dropping casual and Registed as  they sumed to Cnt.

df.drop(['casual','registered'],axis=1,inplace=True)

df.shape
## 3. Data Preparation
df.info()
#### Creating dummy variables for catogorical variables.
# we need to create a dummy variables for four categorical variables. mnth', 'weekday', 'season' & 'weathersit
df = pd.get_dummies(df, drop_first=True)
df.head()
# Divide the train and test data
np.random.seed(0)
df_train, df_test = train_test_split(df, train_size=0.75, test_size=0.25, random_state=100)
df_train.shape

df_test.shape
#### Scaling numeric variables 
#scaling numeric variables of traning data using MinMaxScalor between 0-1.
num_vars =  ['temp', 'atemp', 'hum', 'windspeed','cnt']
scaler=MinMaxScaler()

df_train[num_vars] = scaler.fit_transform(df_train[num_vars])
df_train.head()
df_train.describe()
# From above analysis we can see that numeric variabels has scaled between 0 and 1.

# Plotting a correlation matrix
plt.figure(figsize = (40, 40))
sns.heatmap(df_train.corr(), annot = True, cmap="BuGn")
plt.show()
# poping seperating independent and dependent variables
y_train=df_train.pop('cnt')
X_train=df_train

## 4. Build a Model
####  Model #1 
# Building a model with all features using statsmodels :
import statsmodels.api as sm
X_train_lm = sm.add_constant(X_train)

lr1 = sm.OLS(y_train, X_train_lm).fit()

lr1.params
lr1.summary()
# Here, we can see that all adjusted R-Squire is 0.848 is pretty good.Lets us see if we can reduce the independet features
# Since number of predictor variables is around 30. It will not be feasible to manually eliminate the insignificance variable.
# Will use hybrid (REF + manual) to elimnate insignifiance variable.
# Importing RFE and LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
# Running RFE with the output number of the variable equal to 15
lm = LinearRegression()
lm.fit(X_train, y_train)
rfe = RFE(estimator=lm, n_features_to_select=15)   # running RFE
rfe = rfe.fit(X_train, y_train)
list(zip(X_train.columns,rfe.support_,rfe.ranking_))
# RFE support Variables
col = X_train.columns[rfe.support_]
print(len(col))
col
# RFE not Supported Variables
X_train.columns[~rfe.support_]
# Lets consider only  the RFE supported columns for model building 
X_train_rfe = X_train[col]

 #Model#2 
# Generic function to calculate VIF of variables

def calculateVIF(df):
    vif = pd.DataFrame()
    vif['Features'] = df.columns
    vif['VIF'] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return vif 
calculateVIF(X_train_rfe)
# Model 2 
#Add a constant
X_train_lm2 = sm.add_constant(X_train_rfe)

# Create a first fitted model
lr2 = sm.OLS(y_train, X_train_lm2).fit()
lr2.summary()
# From abvoe analysis all the variables is having low p-values and also adjusted-R-squire value is 0.847 which is been
# increased from 0.843 to 0.847 after droping 14 features, this means most of the features were insignifiance and few were negative collrelatin
# from above VIP table we can see that workingday is having 31.53 vip value which is quite high. lets remove this featues.

X_train_rfe2 = X_train_rfe.drop(["workingday"], axis = 1)
X_train_rfe2.head()
calculateVIF(X_train_rfe2)
# Model 3 
#Add a constant
X_train_lm3 = sm.add_constant(X_train_rfe2)

# Create a first fitted model
lr3 = sm.OLS(y_train, X_train_lm3).fit()
lr3.summary()
# From above we can see that weekday_sat is having high pValue 0.482>0.05  and hum VIP is 14.57>5.
# Lets drop weekday_sat
X_train_rfe3 = X_train_rfe2.drop(["weekday_sat"], axis = 1)
X_train_rfe3.head()

calculateVIF(X_train_rfe3)

# Model 4
#Add a constant
X_train_lm4 = sm.add_constant(X_train_rfe3)

# Create a first fitted model
lr4 = sm.OLS(y_train, X_train_lm4).fit()
lr4.summary()
# we can see that Adj. R-squire is 0.847 and pvalue is less than 0.05 for every features.
# VIP for hum is 14.57>5.so Lets frist remove the hum.

X_train_rfe4 = X_train_rfe3.drop(["hum"], axis = 1)
X_train_rfe4.head()
calculateVIF(X_train_rfe4)

X_train_lm5 = sm.add_constant(X_train_rfe4)

# Create a first fitted model
lr5 = sm.OLS(y_train, X_train_lm5).fit()
lr5.summary()
#From the above  Model Summary Report we can say that all the p-values of respective features are less than 0.05.
# Note: It is a hurestic approach that, if VIF value of any predictor is between 5 & 10, it is recommended to scrutinize such variable from business perspective and analyse any outliers & drop such feature if VIF is greater than 10
# So we can now fix this 
### 5. Linear Regrssion Assumptions Proofs
#### Residual Analysis
# Lets perform Residual Analysis
y_train_pred = lr5.predict(X_train_lm5)
residual = y_train_pred - y_train
#1. Assumption of Normally Distributed Error Term

# Plot the histogram of the error terms
sns.distplot(residual)
plt.xlabel('Residual (Error Term)', fontdict={'fontsize': 20})
plt.title('Error Distribution', fontdict={'fontsize': 25})
plt.show() 

#Inferences: From the above graph it is evident that Error Distribution Is Normallly Distributed Across 0, which indicates that our model has handled the assumption of Error Normal Distribution properly.

#2 Assumption of Error Terms Being Independent

sns.regplot(x=y_train_pred, y=residual)
plt.title('Residual Vs. Predicted Values (Pattern Indentification)', fontdict={'fontsize': 20})
plt.xlabel('Predicted Values', fontdict={'fontsize': 15})
plt.ylabel('Residuals', fontdict={'fontsize': 15})
plt.show()
#From the above graph, we see that there is almost no relation between Residual & Predicted Value
# 3. Homoscedasticity

sns.regplot(x=y_train, y=y_train_pred)
plt.title('Predicted Points Vs. Actual Points', fontdict={'fontsize': 20})
plt.xlabel('Actual Points', fontdict={'fontsize': 15})
plt.ylabel('Predicted Points', fontdict={'fontsize': 15})
plt.show()
# we can see that variance is  similar from both end of fitted line.
# Multicorrelation
calculateVIF(X_train_rfe4)
plt.figure(figsize = (20, 20))
# Heatmap
sns.heatmap(X_train_rfe4.corr(), cmap="YlGnBu", annot=True)
plt.show()
# As per above analysis we didn't found multi-colinearity among independedt variables.
### 6. Model Prediction on Test Data
num_vars = ['temp', 'atemp', 'hum', 'windspeed','cnt']

df_test[num_vars] = scaler.transform(df_test[num_vars])
df_test.head()
df_test.describe()
# divie into x and y
y_test = df_test.pop('cnt')
X_test = df_test
X_test.info()
X_test=X_test[X_train_rfe4.columns]
# Adding constant variable to test dataframe
X_test_lr5 = sm.add_constant(X_test)
X_test_lr5.info()
y_test_pred = lr5.predict(X_test_lr5)

 lr5.params
### 7. Generating R-Square
from sklearn.metrics import r2_score
r2_score(y_test, y_test_pred)
train_adjuested_r_2=0.841
train_r2=0.844
# n is number of rows in X

n = X_test.shape[0]


# Number of features (predictors, p) is the shape along axis 1
p = X_test.shape[1]

# We find the Adjusted R-squared using the formula

test_adjusted_r2 = 1-(1-train_r2)*(n-1)/(n-p-1)
test_adjusted_r2
# Train R-squire=0.844
# Test R-squire=0.7795
# Train Adj-R-squire=0.841
# Test Adj-R-squire=0.832
# That seems a very good models.